# LSH Cascade Search - 基本検証

このノートブックでは、LSH (Locality Sensitive Hashing) を用いた3段階フィルタリング検索の基本動作を検証します。

## 目次
1. データの読み込みと確認
2. SimHashの動作確認
3. HNSW検索 vs LSH Cascade検索
4. パラメータ比較（LSH-4, LSH-8, LSH-16）

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd

from src.lsh import SimHashGenerator, chunk_hash, hamming_distance
from src.db import VectorDatabase
from src.pipeline import LSHCascadeSearcher, HNSWSearcher

## 1. データの読み込みと確認

In [2]:
# DuckDBからデータを読み込み
db_path = Path('../data/sample_vectors_v2.duckdb')  # 手元のWikipedaデータ（１万件）v2: 本文データ
db = VectorDatabase(db_path=db_path)
db.initialize()

print(f'Total documents: {db.count():,}')

# カラム情報を表示
print('\n=== テーブルスキーマ ===')
schema = db.conn.execute('DESCRIBE documents').fetchdf()
print(schema.to_string(index=False))

Total documents: 10,000

=== テーブルスキーマ ===
column_name column_type null key default extra
         id     INTEGER   NO PRI    None  None
       text     VARCHAR  YES NaN    None  None
     vector FLOAT[1024]  YES NaN    None  None
    simhash     VARCHAR  YES NaN    None  None
 lsh_chunks   VARCHAR[]  YES NaN    None  None


In [3]:
# サンプルデータを確認
df_sample = db.get_by_ids([0, 1, 2])
df_sample

,id,text,vector,simhash,lsh_chunks
0,0,{{中華圏の事物\n| 画像=[[File:People's Literature Publ...,"[-0.004686727, -0.01291483, -0.015869895, -0.0...",BB8A8C209B56AA58CD0385D188E61C1F,"[c0_BB, c1_8A, c2_8C, c3_20, c4_9B, c5_56, c6_..."
1,1,モレロス州<noinclude>\n[[Category:Country aliasテンプレ...,"[0.037693564, 0.014720356, -0.008648348, -0.06...",BBB9CE281B56AAF8CB1385598A960E1B,"[c0_BB, c1_B9, c2_CE, c3_28, c4_1B, c5_56, c6_..."
2,2,{{Infobox Musician\n| Name = 碧井...,"[0.027235618, 0.050126687, -0.013588801, -0.06...",B91B8C268316BB7ACA138FDBCA8C081B,"[c0_B9, c1_1B, c2_8C, c3_26, c4_83, c5_16, c6_..."


In [4]:
# # サンプルデータの確認（全カラム表示）
# df_sample = db.get_by_ids([0, 1, 2])
# print(df_sample)
# print("=====")

# print('=== サンプルデータ (3行) ===')
# for i, row in df_sample.iterrows():
#     print(f'\n--- Row {row["id"]} ---')
#     print(f'id: {row["id"]}')
#     print(f'text: {row["text"]}')
#     vec = row["vector"]
#     print(f'vector: [{vec[0]:.6f}, {vec[1]:.6f}, ...] (len={len(vec)})')
#     print(f'simhash: {row["simhash"]}')
#     chunks = row["lsh_chunks"]
#     print(f'lsh_chunks: {chunks[:4]}... (len={len(chunks)})')

## 2. SimHashの動作確認

In [5]:
# SimHashGeneratorの初期化
simhash_gen = SimHashGenerator(dim=1024, hash_bits=128, seed=42)

# サンプルベクトルでSimHashを生成
sample_vec = np.array(df_sample.iloc[0]['vector'], dtype=np.float32)
sample_hash = simhash_gen.hash(sample_vec)

print(f'Vector shape: {sample_vec.shape}')
print(f'SimHash (int): {sample_hash}')
print(f'SimHash (hex): {sample_hash:032X}')

Vector shape: (1024,)
SimHash (int): 249285014298977734719250992799632727071
SimHash (hex): BB8A8C209B56AA58CD0385D188E61C1F


In [6]:
# チャンク分割の確認
for num_chunks in [4, 8, 16]:
    chunks = chunk_hash(sample_hash, num_chunks)
    print(f'LSH-{num_chunks}: {chunks[:4]}... (total {len(chunks)} chunks)')

LSH-4: ['c0_BB8A8C20', 'c1_9B56AA58', 'c2_CD0385D1', 'c3_88E61C1F']... (total 4 chunks)
LSH-8: ['c0_BB8A', 'c1_8C20', 'c2_9B56', 'c3_AA58']... (total 8 chunks)
LSH-16: ['c0_BB', 'c1_8A', 'c2_8C', 'c3_20']... (total 16 chunks)


In [7]:
# ハミング距離の確認
vec1 = np.array(df_sample.iloc[0]['vector'], dtype=np.float32)
vec2 = np.array(df_sample.iloc[1]['vector'], dtype=np.float32)

hash1 = simhash_gen.hash(vec1)
hash2 = simhash_gen.hash(vec2)

dist = hamming_distance(hash1, hash2)
cosine_sim = np.dot(vec1, vec2)

print(f'Hamming distance: {dist} / 128 bits')
print(f'Cosine similarity: {cosine_sim:.4f}')

Hamming distance: 22 / 128 bits
Cosine similarity: 0.7644


## 3. HNSW検索 vs LSH Cascade検索

In [8]:
# 検索器の初期化
hnsw_searcher = HNSWSearcher(db)
lsh_searcher = LSHCascadeSearcher(
    db=db,
    simhash_generator=simhash_gen,
    num_chunks=16,
    # step2_top_n=100,
    step2_top_n=500,
)

In [9]:
# クエリベクトル（最初のドキュメントを使用）
query_vec = np.array(df_sample.iloc[0]['vector'], dtype=np.float32)

# HNSW検索
hnsw_results, hnsw_time = hnsw_searcher.search(query_vec, top_k=10)
print('=== HNSW Results ===')
print(f'Time: {hnsw_time:.2f} ms')
for r in hnsw_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:30]}...')

=== HNSW Results ===
Time: 125.73 ms
  id=0, score=1.0000, text={{中華圏の事物
| 画像=[[File:People's ...
  id=9283, score=0.9133, text={{出典の明記|date=2018年4月17日 (火) 06...
  id=3671, score=0.9106, text={{中華圏の事物
| 画像=[[ファイル:人民教育出版社 -...
  id=6035, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
...
  id=9438, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
...


In [10]:
# LSH Cascade検索
lsh_results, lsh_metrics = lsh_searcher.search(query_vec, top_k=10)
print('=== LSH Cascade Results ===')
print(f'Total time: {lsh_metrics.total_time_ms:.2f} ms')
print(f'Step1 candidates: {lsh_metrics.step1_candidates}')
print(f'Step2 candidates: {lsh_metrics.step2_candidates}')
print()
for r in lsh_results[:5]:
    print(f'  id={r.id}, score={r.score:.4f}, text={r.text[:30]}...')

=== LSH Cascade Results ===
Total time: 410.71 ms
Step1 candidates: 9496
Step2 candidates: 500

  id=0, score=1.0000, text={{中華圏の事物
| 画像=[[File:People's ...
  id=9283, score=0.9133, text={{出典の明記|date=2018年4月17日 (火) 06...
  id=3671, score=0.9106, text={{中華圏の事物
| 画像=[[ファイル:人民教育出版社 -...
  id=9438, score=0.9081, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
...
  id=5861, score=0.9080, text={{中華圏の事物
| 画像=
| 画像の説明=
| 英文=
...


In [11]:
# Recall計算
hnsw_ids = set(r.id for r in hnsw_results)
lsh_ids = set(r.id for r in lsh_results)
recall = len(hnsw_ids & lsh_ids) / len(hnsw_ids)

print(f'Recall@10: {recall:.2f}')

Recall@10: 0.60


In [12]:
print(f'Total docs: {lsh_metrics.total_docs}')
print(f'Step1 candidates: {lsh_metrics.step1_candidates}')
print(f'Step1 reduction: {lsh_metrics.step1_candidates / lsh_metrics.total_docs * 100:.1f}%')


Total docs: 10000
Step1 candidates: 9496
Step1 reduction: 95.0%


In [13]:
# Step1の候補に対して分析
query_hash = simhash_gen.hash(query_vec)
query_chunks = chunk_hash(query_hash, 16)
candidates = db.search_lsh_chunks(query_chunks)

results = []
for _, row in candidates.iterrows():
    doc_hash = int(row['simhash'], 16)
    doc_vec = np.array(row['vector'], dtype=np.float32)
    
    ham_dist = hamming_distance(query_hash, doc_hash)
    cos_sim = np.dot(query_vec, doc_vec)
    
    results.append({
        'id': row['id'],
        'hamming_dist': ham_dist,
        'cosine_sim': cos_sim,
    })

df_analysis = pd.DataFrame(results)

# 相関係数
corr = df_analysis['hamming_dist'].corr(df_analysis['cosine_sim'])
print(f'ハミング距離とコサイン類似度の相関係数: {corr:.3f}')

# HNSW正解のハミング距離順位を確認
df_sorted_ham = df_analysis.sort_values('hamming_dist')
df_sorted_ham['ham_rank'] = range(1, len(df_sorted_ham) + 1)

print('\n=== HNSW正解10件のハミング距離順位 ===')
# hnsw_resultsから動的に取得（セル12で定義済み）
hnsw_ids = set(r.id for r in hnsw_results)
for doc_id in hnsw_ids:
    if doc_id in df_sorted_ham['id'].values:
        row = df_sorted_ham[df_sorted_ham['id'] == doc_id].iloc[0]
        print(f'id={doc_id}: ハミング距離={row["hamming_dist"]}, 順位={row["ham_rank"]}/{len(df_sorted_ham)}, コサイン類似度={row["cosine_sim"]:.4f}')
    else:
        print(f'id={doc_id}: Step1候補に含まれていない')

ハミング距離とコサイン類似度の相関係数: -0.364

=== HNSW正解10件のハミング距離順位 ===
id=0: ハミング距離=0.0, 順位=1.0/9496, コサイン類似度=1.0000
id=7680: ハミング距離=11.0, 順位=2.0/9496, コサイン類似度=0.9048
id=7585: ハミング距離=20.0, 順位=552.0/9496, コサイン類似度=0.8889
id=9283: ハミング距離=15.0, 順位=13.0/9496, コサイン類似度=0.9133
id=4064: ハミング距離=22.0, 順位=1646.0/9496, コサイン類似度=0.8856
id=5861: ハミング距離=18.0, 順位=154.0/9496, コサイン類似度=0.9080
id=6035: ハミング距離=20.0, 順位=709.0/9496, コサイン類似度=0.9081
id=3671: ハミング距離=13.0, 順位=4.0/9496, コサイン類似度=0.9106
id=2459: ハミング距離=24.0, 順位=3284.0/9496, コサイン類似度=0.8835
id=9438: ハミング距離=17.0, 順位=78.0/9496, コサイン類似度=0.9081


In [14]:
# HNSWの正解ID
hnsw_ids = set(r.id for r in hnsw_results)
print(f'HNSW結果のID: {hnsw_ids}')

# Step1の候補ID
query_chunks = chunk_hash(simhash_gen.hash(query_vec), 16)
candidates = db.search_lsh_chunks(query_chunks)
candidate_ids = set(candidates['id'])

# 重複確認
overlap = hnsw_ids & candidate_ids
print(f'Step1候補に含まれるHNSW正解: {len(overlap)}/10')
print(f'含まれているID: {overlap}')
print(f'含まれていないID: {hnsw_ids - candidate_ids}')

HNSW結果のID: {0, 7680, 7585, 9283, 4064, 5861, 6035, 3671, 2459, 9438}
Step1候補に含まれるHNSW正解: 10/10
含まれているID: {0, 7680, 7585, 9283, 4064, 5861, 6035, 3671, 2459, 9438}
含まれていないID: set()


In [15]:
import random

# ランダムに5件のクエリを選択
random.seed(123)
all_docs = db.get_all()
query_indices = random.sample(range(len(all_docs)), 5)

for idx in query_indices:
    query_row = all_docs.iloc[idx]
    query_vec = np.array(query_row['vector'], dtype=np.float32)
    query_id = query_row['id']
    
    print(f'\n{"="*50}')
    print(f'Query ID: {query_id}, Text: {query_row["text"][:30]}...')
    print(f'{"="*50}')
    
    # HNSW検索
    hnsw_results, _ = hnsw_searcher.search(query_vec, top_k=10)
    hnsw_ids = set(r.id for r in hnsw_results)
    
    # Step1候補取得
    query_hash = simhash_gen.hash(query_vec)
    query_chunks = chunk_hash(query_hash, 16)
    candidates = db.search_lsh_chunks(query_chunks)
    
    # 相関分析
    results = []
    for _, row in candidates.iterrows():
        doc_hash = int(row['simhash'], 16)
        doc_vec = np.array(row['vector'], dtype=np.float32)
        ham_dist = hamming_distance(query_hash, doc_hash)
        cos_sim = np.dot(query_vec, doc_vec)
        results.append({'id': row['id'], 'hamming_dist': ham_dist, 'cosine_sim': cos_sim})
    
    df_analysis = pd.DataFrame(results)
    corr = df_analysis['hamming_dist'].corr(df_analysis['cosine_sim'])
    
    # HNSW正解の順位確認
    df_sorted = df_analysis.sort_values('hamming_dist')
    df_sorted['ham_rank'] = range(1, len(df_sorted) + 1)
    
    ranks_in_100 = 0
    ranks_in_500 = 0
    for doc_id in hnsw_ids:
        if doc_id in df_sorted['id'].values:
            rank = df_sorted[df_sorted['id'] == doc_id]['ham_rank'].values[0]
            if rank <= 100:
                ranks_in_100 += 1
            if rank <= 500:
                ranks_in_500 += 1
    
    print(f'Step1候補数: {len(candidates)}')
    print(f'相関係数: {corr:.3f}')
    print(f'HNSW正解がStep1に含まれる: {len(hnsw_ids & set(candidates["id"]))}/10')
    print(f'HNSW正解がTop100に含まれる: {ranks_in_100}/10')
    print(f'HNSW正解がTop500に含まれる: {ranks_in_500}/10')



Query ID: 857, Text: {{基礎情報 アナウンサー
|名前=翠川 秋子
|ふりがな=...
Step1候補数: 8783
相関係数: -0.321
HNSW正解がStep1に含まれる: 10/10
HNSW正解がTop100に含まれる: 2/10
HNSW正解がTop500に含まれる: 4/10

Query ID: 4385, Text: {{Infobox military conflict
| ...
Step1候補数: 8990
相関係数: -0.400
HNSW正解がStep1に含まれる: 9/10
HNSW正解がTop100に含まれる: 4/10
HNSW正解がTop500に含まれる: 6/10

Query ID: 1428, Text: Flag of Queretaro.svg<noinclud...
Step1候補数: 9531
相関係数: -0.655
HNSW正解がStep1に含まれる: 10/10
HNSW正解がTop100に含まれる: 8/10
HNSW正解がTop500に含まれる: 10/10

Query ID: 6672, Text: {{軍隊資料
|名称 = 第33装甲旅団
|画像 = [[F...
Step1候補数: 8941
相関係数: -0.408
HNSW正解がStep1に含まれる: 10/10
HNSW正解がTop100に含まれる: 5/10
HNSW正解がTop500に含まれる: 9/10

Query ID: 4367, Text: {{Pathnav|主題別分類|・・|日本の不動産業|日本の...
Step1候補数: 9210
相関係数: -0.392
HNSW正解がStep1に含まれる: 10/10
HNSW正解がTop100に含まれる: 3/10
HNSW正解がTop500に含まれる: 4/10


In [16]:
# 複数のシード値で比較
seeds = [42, 123, 456, 789, 1000]

# 固定のクエリを使用（最初のドキュメント）
query_row = all_docs.iloc[0]
query_vec = np.array(query_row['vector'], dtype=np.float32)

print(f'Query ID: {query_row["id"]}, Text: {query_row["text"][:30]}...')
print()

for seed in seeds:
    # 新しいシードでSimHashGeneratorを作成
    test_gen = SimHashGenerator(dim=1024, hash_bits=128, seed=seed)
    
    # クエリのハッシュとチャンク
    query_hash = test_gen.hash(query_vec)
    
    # 全ドキュメントのハッシュを再計算して分析
    results = []
    for _, row in all_docs.iterrows():
        doc_vec = np.array(row['vector'], dtype=np.float32)
        doc_hash = test_gen.hash(doc_vec)
        
        ham_dist = hamming_distance(query_hash, doc_hash)
        cos_sim = np.dot(query_vec, doc_vec)
        
        results.append({
            'id': row['id'],
            'hamming_dist': ham_dist,
            'cosine_sim': cos_sim,
        })
    
    df_analysis = pd.DataFrame(results)
    corr = df_analysis['hamming_dist'].corr(df_analysis['cosine_sim'])
    
    # HNSW正解の順位確認
    hnsw_results, _ = hnsw_searcher.search(query_vec, top_k=10)
    hnsw_ids = set(r.id for r in hnsw_results)
    
    df_sorted = df_analysis.sort_values('hamming_dist')
    df_sorted['ham_rank'] = range(1, len(df_sorted) + 1)
    
    ranks_in_100 = sum(1 for doc_id in hnsw_ids 
                       if df_sorted[df_sorted['id'] == doc_id]['ham_rank'].values[0] <= 100)
    ranks_in_500 = sum(1 for doc_id in hnsw_ids 
                       if df_sorted[df_sorted['id'] == doc_id]['ham_rank'].values[0] <= 500)
    
    print(f'Seed={seed}: 相関係数={corr:.3f}, Top100={ranks_in_100}/10, Top500={ranks_in_500}/10')


Query ID: 0, Text: {{中華圏の事物
| 画像=[[File:People's ...

Seed=42: 相関係数=-0.366, Top100=5/10, Top500=6/10
Seed=123: 相関係数=-0.362, Top100=4/10, Top500=5/10
Seed=456: 相関係数=-0.296, Top100=6/10, Top500=9/10
Seed=789: 相関係数=-0.354, Top100=3/10, Top500=5/10
Seed=1000: 相関係数=-0.393, Top100=8/10, Top500=10/10


## 4. パラメータ比較（LSH-4, LSH-8, LSH-16）

In [17]:
# 複数クエリでの比較
import random

# ランダムに10件のクエリを選択
random.seed(42)
all_docs = db.get_all()
query_indices = random.sample(range(len(all_docs)), 10)

results_summary = []

for num_chunks in [4, 8, 16]:
    searcher = LSHCascadeSearcher(
        db=db,
        simhash_generator=simhash_gen,
        num_chunks=num_chunks,
        step2_top_n=100,
    )
    
    recalls = []
    latencies = []
    candidates = []
    
    for idx in query_indices:
        query_vec = np.array(all_docs.iloc[idx]['vector'], dtype=np.float32)
        
        # HNSW baseline
        hnsw_results, _ = hnsw_searcher.search(query_vec, top_k=10)
        hnsw_ids = set(r.id for r in hnsw_results)
        
        # LSH search
        lsh_results, metrics = searcher.search(query_vec, top_k=10)
        lsh_ids = set(r.id for r in lsh_results)
        
        recall = len(hnsw_ids & lsh_ids) / len(hnsw_ids) if hnsw_ids else 0
        recalls.append(recall)
        latencies.append(metrics.total_time_ms)
        candidates.append(metrics.step1_candidates)
    
    results_summary.append({
        'chunks': num_chunks,
        'bits_per_chunk': 128 // num_chunks,
        'avg_recall': np.mean(recalls),
        'avg_latency_ms': np.mean(latencies),
        'avg_candidates': np.mean(candidates),
        'reduction_rate': 1 - np.mean(candidates) / len(all_docs),
    })

df_results = pd.DataFrame(results_summary)
df_results

,chunks,bits_per_chunk,avg_recall,avg_latency_ms,avg_candidates,reduction_rate
0,4,32,0.00,9.577625,0.0,1.00000
1,8,16,0.00,11.625032,0.0,1.00000
2,16,8,0.48,311.887399,8971.6,0.10284


In [18]:
# デバッグ: 単一クエリで4チャンクのマッチングを確認
query_idx = 0
query_vec = np.array(all_docs.iloc[query_idx]['vector'], dtype=np.float32)
query_hash = simhash_gen.hash(query_vec)

print(f'Query ID: {all_docs.iloc[query_idx]["id"]}')
print(f'Query SimHash: {query_hash:032X}')

for num_chunks in [4, 8, 16]:
    query_chunks = set(chunk_hash(query_hash, num_chunks))
    print(f'\n=== {num_chunks}チャンク ===')
    print(f'Query chunks: {query_chunks}')
    
    # 自分自身とマッチするか確認
    doc_hash = int(all_docs.iloc[query_idx]['simhash'], 16)
    doc_chunks = set(chunk_hash(doc_hash, num_chunks))
    print(f'Doc chunks: {doc_chunks}')
    print(f'一致: {query_chunks & doc_chunks}')
    
    # 候補数をカウント
    match_count = 0
    for _, row in all_docs.head(100).iterrows():
        doc_hash = int(row['simhash'], 16)
        doc_chunks = set(chunk_hash(doc_hash, num_chunks))
        if query_chunks & doc_chunks:
            match_count += 1
    print(f'最初の100件中のマッチ数: {match_count}')


Query ID: 0
Query SimHash: BB8A8C209B56AA58CD0385D188E61C1F

=== 4チャンク ===
Query chunks: {'c0_BB8A8C20', 'c3_88E61C1F', 'c2_CD0385D1', 'c1_9B56AA58'}
Doc chunks: {'c0_BB8A8C20', 'c3_88E61C1F', 'c2_CD0385D1', 'c1_9B56AA58'}
一致: {'c0_BB8A8C20', 'c3_88E61C1F', 'c2_CD0385D1', 'c1_9B56AA58'}
最初の100件中のマッチ数: 1

=== 8チャンク ===
Query chunks: {'c0_BB8A', 'c6_88E6', 'c5_85D1', 'c2_9B56', 'c3_AA58', 'c1_8C20', 'c4_CD03', 'c7_1C1F'}
Doc chunks: {'c0_BB8A', 'c6_88E6', 'c5_85D1', 'c2_9B56', 'c3_AA58', 'c1_8C20', 'c4_CD03', 'c7_1C1F'}
一致: {'c0_BB8A', 'c6_88E6', 'c5_85D1', 'c2_9B56', 'c3_AA58', 'c1_8C20', 'c4_CD03', 'c7_1C1F'}
最初の100件中のマッチ数: 22

=== 16チャンク ===
Query chunks: {'c5_56', 'c0_BB', 'c4_9B', 'c12_88', 'c13_E6', 'c1_8A', 'c11_D1', 'c7_58', 'c14_1C', 'c15_1F', 'c9_03', 'c8_CD', 'c3_20', 'c2_8C', 'c10_85', 'c6_AA'}
Doc chunks: {'c5_56', 'c0_BB', 'c4_9B', 'c12_88', 'c13_E6', 'c1_8A', 'c11_D1', 'c7_58', 'c14_1C', 'c15_1F', 'c9_03', 'c8_CD', 'c3_20', 'c2_8C', 'c10_85', 'c6_AA'}
一致: {'c5_56', 'c0_BB'

In [19]:
# データベース接続を閉じる
db.close()
print('Done!')

Done!


## 5. v2データセット評価レポート

### 主要結果

| 指標 | 値 | 評価 |
|------|-----|------|
| **Recall@10** | 0.60 | 改善の余地あり |
| **相関係数** | -0.364 | 弱い（理論値は-1.0に近いはず） |
| **Step1正解含有率** | 10/10 | 良好 |
| **Step1削減率** | 5%のみ（9496/10000残存） | 課題 |

### 根本的な問題

**高次元空間でのコサイン類似度の集中現象**
- ランダムペアの類似度: 平均0.775、標準偏差0.030
- 全データが0.65〜1.0の狭い範囲に集中
- この狭い範囲での区別にSimHash 128bitでは不十分

### 良い点

1. **Step1で正解が漏れていない**（10/10含有）

2. **一部のクエリは高性能**
   - Query 1428 (Template): Top500に10/10含有、相関 -0.655
   - Query 6672 (装甲旅団): Top500に9/10含有

### 課題

1. **ハミング距離順位のばらつき**
   ```
   id=3671: 順位4    → 良好
   id=2459: 順位3284 → 問題
   id=4064: 順位1646 → 問題
   ```

2. **シードによる変動が大きい**
   ```
   Seed=1000: Top500=10/10
   Seed=789:  Top500=5/10
   ```

3. **ビット数を増やしても限界がある**（02_v2_lsh_accuracy_analysis.ipynbより）
   - 128bit: -0.366
   - 256bit: -0.514
   - 512bit: -0.566（頭打ち傾向）

### 次のステップ候補

| 優先度 | アプローチ | 期待効果 |
|--------|-----------|---------|
| 1 | `step2_top_n`を1000〜2000に増加 | Recall向上（簡単だが対症療法） |
| 2 | 複数シードのアンサンブル | 安定性向上 |
| 3 | ハッシュビット数256bitに増加 | 相関 -0.364 → -0.514 程度 |
| - | **根本解決**: 異なるLSH手法の検討 | 要調査 |